In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
import sys
from keras.callbacks import EarlyStopping
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Reshape, Attention, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Reshape, Flatten, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model

2024-07-12 12:19:25.506956: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 12:19:25.576801: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-12 12:19:25.655136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 12:19:25.708761: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 12:19:25.724273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 12:19:25.828443: I tensorflow/core/platform/cpu_feature_gu

In [27]:
from tensorflow.keras.layers import Input, GRU, TimeDistributed, Dense, Dropout, Attention, Concatenate, Flatten
from tensorflow.keras.regularizers import l2

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
final_arousal_df = pd.read_csv('Extracted_Features/Final_Arousal_Features.csv')
final_valence_df = pd.read_csv('Extracted_Features/Final_Valence_Features.csv')

In [4]:
final_valence_df.drop(["Arousal","Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)
final_arousal_df.drop(["Valence", "Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)

In [5]:
# Function to convert string representation of list to a list of floats
def str_to_float_list(s):
    return list(map(float, s.strip('[]').split()))

In [6]:
columns_to_convert = []
for col in final_valence_df.columns:
    if final_valence_df[col].dtype == 'object' and col != 'Target_Emotion':
        columns_to_convert.append(col)

In [7]:
for col in columns_to_convert:
    final_valence_df[col] = final_valence_df[col].apply(str_to_float_list)

In [8]:
final_valence_array = final_valence_df.to_numpy()

In [10]:
np.save("Extracted_Features/Final_Valence_array.npy",final_valence_array)

In [13]:
final_valence_array.shape

(414, 158)

In [14]:
X = final_valence_df.drop(columns=['Valence', 'Target_Emotion'])  # Drop the target columns
y = final_valence_df['Valence']  # Target column

In [15]:
def flatten_features(X):
    return np.array(X.apply(lambda row: np.hstack(row.values), axis=1).tolist())

X_flattened = flatten_features(X)

## Random Forest Regressor

In [25]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42)

# Model training with Random Forest Regressor
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [26]:
# Predictions and evaluation
y_pred = model.predict(X_test)

In [27]:
mse = mean_squared_error(y_test, y_pred)

In [28]:
r2 = r2_score(y_test, y_pred)

In [29]:
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 1.8003421686746985
R^2 Score: -0.03752360716078251


## Using Conv1D 

In [16]:
# Splitting the features (X) and target (y)
X = final_valence_df.drop(columns=['Valence', 'Target_Emotion'])  # Drop the target columns
y = final_valence_df['Valence']  # Target column

In [21]:
len(final_valence_df.columns)

158

In [20]:
list_columns = [col for col in final_valence_df.columns if isinstance(final_valence_df[col].iloc[0], list)]
len(list_columns)

140

In [22]:
def list_len(x):
    if isinstance(x, list):
        return len(x)
    else:
        return 0  # Or any other default value

In [23]:
max_length = final_valence_df[list_columns].applymap(list_len).max().max()
print("Maximum length of lists:", max_length)

Maximum length of lists: 640


/tmp/ipykernel_5334/1240711617.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_length = final_valence_df[list_columns].applymap(list_len).max().max()


In [24]:
def pad_list(lst, max_length):
    return np.pad(lst, (0, max_length - len(lst)), 'constant')

for col in list_columns:
    final_valence_df[col] = final_valence_df[col].apply(lambda x: pad_list(x, max_length))

In [25]:
def convert_to_3d_array(df, columns):
    return np.stack(df[columns].apply(lambda row: np.stack(row.values), axis=1).values)

# Prepare the data
X = convert_to_3d_array(final_valence_df, list_columns)
y = final_valence_df['Valence'].values

In [26]:
print(X.shape)  # Should be (number_of_samples, number_of_columns, max_length)
print(y.shape)  # Should be (number_of_samples,)

(414, 140, 640)
(414,)


In [28]:
def get_model(num_time_steps, num_features, _rnn_nb, _fc_nb, dropout_rate=0.5, l2_reg=1e-4):
    spec_start = Input((num_time_steps, num_features))

    spec_x = spec_start

    for _r in _rnn_nb:
        spec_x = GRU(_r, activation='tanh', dropout=dropout_rate, recurrent_dropout=dropout_rate, return_sequences=True, kernel_regularizer=l2(l2_reg))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)

    attention = Attention()([spec_x, spec_x])

    attended = Attention()([spec_x, attention])

    spec_x = Concatenate(axis=-1)([spec_x, attended])

    for _f in _fc_nb:
        spec_x = TimeDistributed(Dense(_f, activation='relu', kernel_regularizer=l2(l2_reg)))(spec_x)
        spec_x = Dropout(dropout_rate)(spec_x)

    out = TimeDistributed(Dense(1, activation='linear'))(spec_x)
    out = Flatten()(out)
    out = Dense(1, activation='linear')(out)

    _model = Model(inputs=spec_start, outputs=out)
    _model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    _model.summary()

    return _model


In [29]:
# Example usage
num_time_steps = len(list_columns)  # Number of feature columns
num_features = max_length  # Length of each list after padding
model = get_model(num_time_steps, num_features, _rnn_nb=[64, 128], _fc_nb=[64])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 140, 640)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ (None, 140, 64)   │    135,552 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 140, 64)   │          0 │ gru[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 140, 128)  │     74,496 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 140, 128)  │          0 │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 140, 128)  │          0 │ dropout_1[0][0],  │
│ (Attention)         │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 140, 128)  │          0 │ dropout_1[0][0],  │
│ (Attention)         │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 140, 256)  │          0 │ dropout_1[0][0],  │
│ (Concatenate)       │                   │            │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 140, 64)   │     16,448 │ concatenate[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 140, 64)   │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 140, 1)    │         65 │ dropout_2[0][0]   │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 140)       │          0 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        141 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 226,702 (885.55 KB)

 Trainable params: 226,702 (885.55 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2024-07-12 14:34:22.328973: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 148377600 exceeds 10% of free system memory.


13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 360ms/step - loss: nan - mean_squared_error: nan
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 351ms/step - loss: nan - mean_squared_error: nan
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 350ms/step - loss: nan - mean_squared_error: nan
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 346ms/step - loss: nan - mean_squared_error: nan
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 348ms/step - loss: nan - mean_squared_error: nan
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 349ms/step - loss: nan - mean_squared_error: nan
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 343ms/step - loss: nan - mean_squared_error: nan
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 346ms/step - loss: nan - mean_squared_error: nan
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 342ms/step - loss: nan - mean_squared_error: nan
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 344ms/step - loss: nan - mean_squared_error: nan
